In [18]:
import logging
import openpyxl
import re
import traceback
import numpy as np
import pandas as pd
import requests as rq
from collections import defaultdict
from random import randint

### Define Constants

In [4]:
GOOGLE_SHEET_LINK = 'https://docs.google.com/spreadsheets/d/1u9xez7cCm2zTK65cn1WrtmKDK2FuqfckQ44PxGXfmLc/edit#gid=1053414731'

EXCEL_FILENAME = 'test-cases/google_sheet_excel.xlsx'

### if you have an excel file already
### set HAVE_EXCEL_FILE variable to True and set EXCEL_FILE_PATH to the path of the local excel file
HAVE_EXCEL_FILE = True
EXCEL_FILE_PATH = 'test-cases/daily_transactions.xlsx'

SQL_IMPLEMENTATION = 'sqlserver' ### postgres, mysql, sqlserver

SHOW_PRINT = False

ERROR_LOG_STRING_FORMAT = '''
    {section_title}
    ---
    {description}
    ---
    {error_message}
''' + '='*60 + '\n\n'

In [5]:
INCONSISTENT_COLUMNS = defaultdict(
    lambda : {'datetime' : 'datetime'}, ### change this to date/datetime based on what is required
    {
        'postgres': {'datetime' : 'timestamp'}
    }
)

DTYPES_MAPPING = {
    'object' : 'varchar(255)',
    'int' : 'int',
    'float' : 'float'
}

DTYPES_MAPPING = {**DTYPES_MAPPING, **INCONSISTENT_COLUMNS[SQL_IMPLEMENTATION]}
if SHOW_PRINT:
    print(DTYPES_MAPPING)

### Logging Helper

In [7]:
def error_logging_helper(log_filename:str = 'all_logs.log', logger_name:str = 'error_logger'):
    '''
    helper function for logging errors
    '''
    logger = logging.getLogger(logger_name)
    handler = logging.FileHandler(
        filename=log_filename,
        mode='a+'
    )
    log_format = logging.Formatter('%(asctime)s - %(message)s')
    handler.setLevel(logging.INFO)
    handler.setFormatter(log_format)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    print(logger.getEffectiveLevel())
    
    return logger

logger = error_logging_helper('all_logs.log')

20


### Download Google Sheet as Excel

In [8]:
def download_xlsx_from_google_sheet():
    '''
    function for downloading google sheet as xlsx file
    
    Returns: boolean value based on success/failure
    '''
    return_bool_value = True
    
    if 'docs.google.com' not in GOOGLE_SHEET_LINK and 'spreadsheets' not in GOOGLE_SHEET_LINK:
        logger.error(
            ERROR_LOG_STRING_FORMAT.format(
                section_title = 'Error occured in downloading google sheet. Please check google sheet link',
                description = GOOGLE_SHEET_LINK,
                error_message = GOOGLE_SHEET_LINK
            )
        )
        return_bool_value = False
    else:
        try:
            ### extract id and try downloading google sheet
            google_sheet_id = re.search(
                pattern = r'(\/d\/)(\w+)(\/)',
                string = GOOGLE_SHEET_LINK
            ).group(2)

            sheet_download_rq = rq.get(
                url = 'https://docs.google.com/spreadsheets/u/0/d/' + google_sheet_id + "/export",
                params = {
                    'format' : 'xlsx',
                    'id' : google_sheet_id
                },
                stream = True
            )
    
            ### if permission error log error else create xlsx file
            if 'sign in' in sheet_download_rq.text.lower():
                logger.error(
                    ERROR_LOG_STRING_FORMAT.format(
                        section_title = 'Error occured in downloading google sheet. Please check google sheet link',
                        description = GOOGLE_SHEET_LINK,
                        error_message = sheet_download_rq.url
                    )
                )
                return_bool_value = False
            else:
                with sheet_download_rq as f:
                    with open(EXCEL_FILENAME, 'wb') as output:
                        output.write(f.content)
                    
                return_bool_value = True
            
        except Exception as e:
            logger.error(
                ERROR_LOG_STRING_FORMAT.format(
                    section_title = 'Error occured in downloading google sheet. Please check google sheet link: ' + GOOGLE_SHEET_LINK,
                    description = str(e),
                    error_message = str(traceback.format_exc())
                )
            )
            
            return_bool_value = False
            
    return return_bool_value

### Open Downloaded Excel and Convert to DataFrame

In [9]:
def convert_sheets_to_dfs(workbook:openpyxl.workbook.workbook.Workbook, sheet_name:str):
    '''
    takes the sheetname and openpyxl workbook and returns a dataframe 
    and dataypes of each column
    
    Inputs:
        workbook: openpyxl workbook variable obtained using openpyxl.load_workbook()
        sheet_name: name of the sheet
    
    Returns: dataframe df and df.dtypes
    '''
    sheet_data = workbook[sheet_name].values
    df = pd.DataFrame(sheet_data, columns = next(sheet_data))
    df = df.dropna(axis=1, how='all').dropna(axis=0, how='all')
    
    if SHOW_PRINT:
        print(df.head(3), end="\n\n")
        print(df.dtypes, end="\n\n")
    
    return df, df.dtypes

### Helper Functions for Generating Queries

In [10]:
def generate_create_table_query(dtype_df:pd.core.series.Series, table_name:str):
    '''
    function for generating the create ddl query
    
    Inputs:
        - dtype_df: pandas series with dtypes obtained form the `convert_sheets_to_dfs` function
        - table_name: name of the table to be created
        
    Returns:
        - string with create ddl query
    '''
    
    create_table_structure = '''
    create table {table_name} (
    {columns_dtypes}
    );
    '''
    
    complete_colname_dtype_string = ''

    for col, dtype in dtype_df.iteritems():
        cleaned_dtype = re.search(r'([a-zA-Z]+)', str(dtype)).group()
        colname_dtype_string = '\t' + col + ' ' + DTYPES_MAPPING[cleaned_dtype] + ',\n'
        complete_colname_dtype_string += colname_dtype_string

    complete_colname_dtype_string = complete_colname_dtype_string.rsplit(
        sep=',',
        maxsplit=1
    )[0]
    
    create_table_query_populated = create_table_structure.format(
        table_name = table_name,
        columns_dtypes = complete_colname_dtype_string
    )
    
    print(create_table_query_populated)
    
    return create_table_query_populated

In [20]:
def generate_insert_values_query(df:pd.DataFrame, table_name:str):
    '''
    function for generating the `insert into table values` query
    
    Inputs:
        - df: pandas dataframe obtained from the `convert_sheets_to_dfs` function
        - table_name: name of the table to be created
        
    Returns:
        - string with insert query
    '''
    
    insert_query_structure = '''
    insert into 
        {table_name} ({col_names}) 
    values
    {values_query_string}
    ;
    
    select * from {table_name};
    '''
    
    ### create a new col in the dataframe with all other columns concatenated as string
    df['final_col'] = ''
    
    for col, dtype in zip(df.dtypes.index, df.dtypes.values):
        if col != 'final_col':
            if dtype in [float, np.float64] or dtype in [int, np.int64]:
                df['final_col'] += df[col].astype(str) + ", "
            else:
                df['final_col'] += "'" + df[col].astype(str) + "', "
            
    df['final_col'] = df['final_col'].apply(lambda x: "\t(" + x.rsplit(sep=',', maxsplit=1)[0] + ")")

    insert_values_query_string = ',\n'.join(df['final_col'].values)
    colnames_string = ', '.join(df.drop('final_col', axis=1).dtypes.index)
    
    insert_query_populated = insert_query_structure.format(
        table_name = table_name,
        col_names = colnames_string,
        values_query_string = insert_values_query_string
    )
    
    print(insert_query_populated)
    
    return insert_query_populated

### Connect Everything Together

In [23]:
if not HAVE_EXCEL_FILE:
    download_check = download_xlsx_from_google_sheet()
else:
    download_check = True

if download_check:
    if not HAVE_EXCEL_FILE:
        wb = openpyxl.load_workbook(EXCEL_FILENAME, data_only=True)
    else:
        wb = openpyxl.load_workbook(EXCEL_FILE_PATH, data_only=True)
        
    print(wb.sheetnames)
    
    for sheet_name in wb.sheetnames:
        ### clean up sheet name to make a valid table name
        table_name = sheet_name.replace(' ', '')
        table_name = ''.join(
            re.findall('(\w+)', table_name)
        )
        if table_name == '':
            table_name = 'table_'+str(randint(1, 100000))

        print("###"*15, " ", sheet_name, ":", table_name, " ", "###"*15)

        df, dtype_df = convert_sheets_to_dfs(wb, sheet_name)

        print("~~~"*15)
        create_table_query_populated = generate_create_table_query(dtype_df, table_name)
        insert_query_populated = generate_insert_values_query(df, table_name)
        print("~~~"*15)

        print("###"*35, end="\n\n")
else:
    print(download_check)

['transaction']
#############################################   transaction : transaction   #############################################
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    create table transaction (
    	transaction_date datetime,
	user_id int,
	card_id int,
	amount float,
	transaction_type varchar(255),
	merchant_city varchar(255),
	merchant_state varchar(255)
    );
    

    insert into 
        transaction (transaction_date, user_id, card_id, amount, transaction_type, merchant_city, merchant_state) 
    values
    	('2013-05-01', 0, 0, 35.76, 'Swipe Transaction', 'Monterey Park', 'CA'),
	('2016-12-25', 0, 2, 138.14, 'Chip Transaction', 'Mira Loma', 'CA'),
	('2016-10-01', 0, 3, 26.25, 'Online Transaction', 'ONLINE', 'ONLINE'),
	('2009-08-29', 0, 3, 23.5, 'Swipe Transaction', 'La Verne', 'CA'),
	('2015-05-31', 0, 0, 1.67, 'Chip Transaction', 'La Verne', 'CA'),
	('2006-10-06', 0, 3, 137.18, 'Swipe Transaction', 'La Verne', 'CA'),
	('2012-11-26', 0, 2, 7.29, 'Swipe Tra

C:\Users\frank\AppData\Local\Temp\ipykernel_17344\977948207.py:21: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col, dtype in dtype_df.iteritems():


In [13]:
wb